In [506]:
import glob


files = glob.glob('*/conf/*.yaml')
print(len(files))
files

144


['scenario_alpha_battle/conf/mles_params.yaml',
 'scenario_alpha_battle/conf/random_params.yaml',
 'scenario_alpha_battle/conf/agg_features_params.yaml',
 'scenario_alpha_battle/conf/embeddings_validation.yaml',
 'scenario_alpha_battle/conf/rtd_params.yaml',
 'scenario_alpha_battle/conf/nsp_params.yaml',
 'scenario_alpha_battle/conf/embeddings_validation_baselines_unsupervised.yaml',
 'scenario_alpha_battle/conf/cpc_params.yaml',
 'scenario_alpha_battle/conf/embeddings_validation_short.yaml',
 'scenario_alpha_battle/conf/sop_params.yaml',
 'scenario_alpha_battle/conf/barlow_twins_params.yaml',
 'scenario_alpha_battle/conf/dataset_inference_file.yaml',
 'scenario_bowl2019/conf/pl_fit_target.yaml',
 'scenario_bowl2019/conf/mles_params.yaml',
 'scenario_bowl2019/conf/cpc_v2_params.yaml',
 'scenario_bowl2019/conf/random_params.yaml',
 'scenario_bowl2019/conf/agg_features_params.yaml',
 'scenario_bowl2019/conf/embeddings_validation.yaml',
 'scenario_bowl2019/conf/rtd_params.yaml',
 'scenari

## TODO:

1) trx_encoder block shifted 2 spaces right

In [507]:
test_file_name = 'scenario_age_pred/conf/test.yaml'

In [508]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    for ix, line in enumerate(lines):
        if start and line[4] != ' ' and not (line[:13] == '    positions' or
                                             line[:14] == '    embeddings' or
                                             line[:18] == '    numeric_values' or
                                             line[:19] == '    norm_embeddings' or
                                             line[:20] == '    embeddings_noise' or
                                             line[:22] == '    clip_replace_value' or
                                             line[:28] == '    use_batch_norm_with_lens'):
#             print(line)
            start = False
            
        if line[:14] == '  trx_encoder:':
            lines[ix] = '  {l}'.format(l=line)
            start = True
        elif start:
            lines[ix] = '  {l}'.format(l=line)
            changed = True
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

66 files changed


2) add target to trxencoder

In [509]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
all_files.remove('scenario_age_pred/conf/mles_params.yaml')
all_files.remove('scenario_age_pred/conf/cpc_params.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for ix, line in enumerate(lines):
        if line[:16] == '    trx_encoder:':
            lines.insert(ix + 1, '      _target_: ptls.trx_encoder.TrxEncoder\n')
            cnt += 1
            break

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

67 files changed


3) encoder_type -> seq_encoder + _target_

In [510]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
all_encoder_types = set()
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    for ix, line in enumerate(lines):
        if line.find('encoder_type') != -1:
            ix = line.find(':')
            type_name = line[ix + 2: -1]
            all_encoder_types.add(type_name)

        
all_encoder_types

{'agg_features',
 'distribution_targets',
 'emb_valid',
 'pretrained',
 'rnn',
 'statistics',
 'transf'}

In [511]:
m = {'agg_features': 'ptls.seq_encoder.rnn_encoder.AggFeatureSeqEncoder',
     'rnn': 'ptls.seq_encoder.rnn_encoder.RnnSeqEncoder',
     'distribution_targets': 'ptls.seq_to_target.SequenceToTarget',
     'emb_valid': 'ptls.seq_encoder.dummy_encoder.DummyEncoder',
     'statistics': 'ptls.seq_encoder.statistics_encoder.StatisticsEncoder',
     'transf': 'ptls.seq_encoder.transf_seq_encoder.TransfSeqEncoder'
}

In [512]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    start = False
    for i, line in enumerate(lines):
        enc_type_ix = line.find('encoder_type')
        if enc_type_ix != -1:
            ix = line.find(':')
            type_name = line[ix + 2: -1]
            if type_name != 'pretrained':
                lines[i] = ' ' * enc_type_ix + 'seq_encoder:\n'
                lines.insert(i + 1, ' ' * enc_type_ix + f'  _target_: {m[type_name]}\n')
                cnt += 1
                break


    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

69 files changed


### Both rnn: and transf:

In [513]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    rnn_found = transf_found = False
    for i, line in enumerate(lines):
        if line.find('rnn:') != -1:
            rnn_found = True
        if line.find('transf:') != -1:
            transf_found = True
    if rnn_found and transf_found:
        cnt += 1
        print(config_fname)

print(f'{cnt} files changed')

scenario_alpha_battle/conf/mles_params.yaml
scenario_alpha_battle/conf/barlow_twins_params.yaml
scenario_age_pred/conf/mles_params_for_finetuning.yaml
scenario_x5/conf/mles_params.yaml
scenario_x5/conf/mles_params_for_finetuning.yaml
scenario_x5/conf/barlow_twins_params.yaml
6 files changed


### Delete transf: block where rnn: is presented

In [514]:
both_files = ['scenario_alpha_battle/conf/mles_params.yaml',
              'scenario_alpha_battle/conf/barlow_twins_params.yaml',
              'scenario_age_pred/conf/mles_params_for_finetuning.yaml',
              'scenario_x5/conf/mles_params.yaml',
              'scenario_x5/conf/mles_params_for_finetuning.yaml',
              'scenario_x5/conf/barlow_twins_params.yaml']

cnt = 0

# all_files = [test_file_name]
all_files = both_files
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    yyy = 0
    start_ix = None
    start_trx = False
    start_trx_ix = None
    for i, line in enumerate(lines.copy()):
        ix = line.find('transf:')
        if ix != -1:
            start_ix = ix
            start = True
            lines.pop(i + yyy)
            yyy -= 1
        elif start and line[start_ix] != ' ':
            start = False
            break
        elif start:
            nnn = line.find('trx_encoder:')
            if start_trx or nnn != -1:
                start_trx = True
                lines[i + yyy] = line[2:]
                continue
            lines.pop(i + yyy)
            yyy -= 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{len(both_files)} files changed')

6 files changed


4) remove rnn: and transf:

In [515]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    for i, line in enumerate(lines):
        if line.find('rnn:') != -1 or line.find('transf:') != -1:
            lines.pop(i)
            changed = True
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

60 files changed


5) data_module_class

In [516]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    for i, line in enumerate(lines):
        if line.find('data_module_class:') != -1:
            pos = line.find(':')
            changed = True
            data_module_class_name = line[pos + 2: -1]
            lines.pop(i)
            break
    if changed:
#         print(data_module_class_name)
        cnt += 1
    
        for i, line in enumerate(lines):
            ix = line.find('data_module:')
            if ix != -1:
                lines.insert(i + 1, ' ' * ix + f'  _target_: {data_module_class_name}\n')
                break

                

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

58 files changed


6) params -> pl_module

In [517]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    for i, line in enumerate(lines):
        if line[:7] == 'params:':
            lines[i] = 'pl_module:\n'
            changed = True
            break
    if changed:
#         print(data_module_class_name)
        cnt += 1
          

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

93 files changed


7) spaces before blocks

In [518]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
all_files.remove('scenario_age_pred/conf/mles_params.yaml')
all_files.remove('scenario_age_pred/conf/cpc_params.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    ii = 0
    for i, line in enumerate(lines.copy()):
        if line[0] != ' ' and i > 0:
            lines.insert(i + ii, '\n')
            changed = True
            ii += 1

        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

129 files changed


8) handle pretrained_encoder

In [519]:
cnt = 0

all_params = set()

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    found_start_ix = None
    dd = {}
    for i, line in enumerate(lines.copy()):
        ix = line.find('pretrained:')
        if ix != -1:
            changed = start = True
            found_start_ix = ix
            continue
        if start and line[found_start_ix] != ' ':
            start = False
        elif start:
            dd[line[:line.find(':')].strip()] = line[line.find(':') + 2: -1]
#             all_params.add(line[:line.find(':')].strip())
#             print(line[line.find(':') + 2: -1])

    changed = start = False
    found_start_ix = None
    ii = 0
    for i, line in enumerate(lines.copy()):
        if line[:10] == 'pl_module:' and 'model_path' in dd:
            lines.insert(i + ii, f'pretrained_encoder_path: {dd["model_path"]}\n')
            lines.insert(i + 1 + ii, f'pretrained_module_cls: {dd["pl_module_class"]}\n\n')
            ii += 3
        
        
        ix = line.find('pretrained:')
        if ix != -1:
            changed = start = True
            found_start_ix = ix
            ii -= 1
            lines.pop(i + ii)
            continue
        if start and line[found_start_ix] != ' ':
            start = False
            if 'lr' in dd:
                lines.insert(i + ii - 1, ' ' * found_start_ix + f'pretrained_lr: {dd["lr"]}\n')
        elif start:
            ii -= 1
            lines.pop(i + ii)
                         
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

24 files changed


In [520]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    ii = 0
    for i, line in enumerate(lines.copy()):
        if line[:22] == 'pretrained_module_cls:':
            changed = True
            ixxx = line.find(':')
            module_name = line[ixxx+2:-1]
            lines[i] = 'pretrained_module_cls:\n'
            lines.insert(i + ii + 1, '  _target_: hydra.utils.get_class\n')
            lines.insert(i + ii + 2, f'  path: {module_name}\n')
            ii += 2
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

24 files changed


In [521]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    ii = 0
    for i, line in enumerate(lines.copy()):
        if line.find('encoder_type: pretrained') != -1:
            changed = True  
            lines.pop(i + ii)
            ii -= 1
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

24 files changed


9) pl_module_class -> _target_

In [522]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('pl_module_class:')
        if ix != -1:
            changed = True
            pl_module_class_name = line[line.find(':') + 2: -1]
            lines[i] = ' ' * ix + f'_target_: {pl_module_class_name}\n'
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

58 files changed


10) _target_ for pl_files and score_metric -> metric_list

In [523]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('pl_fit_') == -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('data_module:')
        if ix != -1:
            changed = True
            lines.insert(i + 1, '  _target_: ptls.data_load.data_module.cls_data_module.ClsDataModuleTrain\n')
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

35 files changed


In [524]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('pl_fit_') == -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('pl_module:')
        if ix != -1:
            changed = True
            lines.insert(i + 1, '  _target_: ptls.seq_to_target.SequenceToTarget\n')
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

35 files changed


In [525]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('pl_fit_') == -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('score_metric: auroc')
        if ix != -1:
            changed = True
            lines[i] = ' ' * ix + 'metric_list:\n'
            lines.insert(i + 1, '  - auroc\n')
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

6 files changed


In [526]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('pl_fit_') == -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('score_metric: accuracy')
        if ix != -1:
            changed = True
            lines[i] = ' ' * ix + 'metric_list:\n'
            lines.insert(i + 1, ' ' * ix + '  - accuracy\n')
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

12 files changed


In [527]:
cnt = 0

# all_files = glob.glob('*/conf/cpc_v2_pl_fit_finetuning.yaml')
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('pl_fit_') == -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('score_metric:')
        if ix != -1:
            changed = True
            lines[i] = ' ' * ix + 'metric_list:\n'
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

17 files changed


In [528]:
cnt = 0

# all_files = glob.glob('*/conf/cpc_v2_pl_fit_finetuning.yaml')
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('pl_fit_') == -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    start_ix = None
    yy = 0
    for i, line in enumerate(lines.copy()):
        ix = line.find('metric_list:')
        if ix != -1:
            start = True
            start_ix = ix
            continue
        if start and line[start_ix] != ' ':
            start = False
        elif start:
            changed = True
            ff_ix = line.find('-')
            if line.find('auroc') != -1:
                lines[i + yy] = ' ' * ff_ix + 'auroc:\n'
                lines.insert(i + yy + 1, ' ' * (ff_ix + 2) + '_target_: torchmetrics.AUROC\n')
                lines.insert(i + yy + 2, ' ' * (ff_ix + 2) + 'num_classes: 2\n')
                lines.insert(i + yy + 3, ' ' * (ff_ix + 2) + 'compute_on_step: false\n')
                yy += 3
            elif line.find('accuracy') != -1:
                lines[i + yy] = ' ' * ff_ix + 'accuracy:\n'
                lines.insert(i + yy + 1, ' ' * (ff_ix + 2) + '_target_: torchmetrics.Accuracy\n')
                lines.insert(i + yy + 2, ' ' * (ff_ix + 2) + 'compute_on_step: false\n')
                yy += 2        
            

    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

29 files changed


11) remove 'score_metric' from other than pl_ files

In [529]:
cnt = 0

# all_files = glob.glob('*/conf/cpc_v2_pl_fit_finetuning.yaml')
all_files = glob.glob('*/conf/*.yaml')
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('score_metric:')
        if ix != -1:
            changed = True
            lines.pop(i)
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

4 files changed


12) validation_metric_params

In [530]:
cnt = 0

# all_files = glob.glob('*/conf/cpc_v2_pl_fit_finetuning.yaml')
all_files = glob.glob('*/conf/*.yaml')
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('validation_metric_params:')
        if ix != -1:
            changed = True
            lines[i] = ' ' * ix + 'validation_metric:\n'
            lines.insert(i + 1, ' ' * (ix + 2) + '_target_: ptls.metric_learn.metric.BatchRecallTopPL\n')
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

30 files changed


13) delete head block: from 

scenario_bowl2019/conf/pl_fit_target.yaml

scenario_rosbank/conf/pl_fit_target.yaml

scenario_bowl2019/conf/rtd_params.yaml

scenario_rosbank/conf/rtd_params.yaml

scenario_age_pred/conf/rtd_params.yaml

In [531]:
cnt = 0

# all_files = ['scenario_age_pred/conf/test.test']
all_files = ['scenario_bowl2019/conf/pl_fit_target.yaml',
             'scenario_rosbank/conf/pl_fit_target.yaml',
             'scenario_bowl2019/conf/rtd_params.yaml',
             'scenario_rosbank/conf/rtd_params.yaml',
             'scenario_age_pred/conf/rtd_params.yaml']
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    start_ix = None
    yyy = 0
    for i, line in enumerate(lines.copy()):
        ix = line.find('head:')
        if ix != -1:
            changed = True
            start = True
            start_ix = ix
            lines.pop(i + yyy)
            yyy -= 1
        elif start and line[start_ix] != ' ':
            start = False
        elif start:
            lines.pop(i + yyy)
            yyy -= 1
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

5 files changed


14) head_layers

In [532]:
cnt = 0

# all_files = ['scenario_age_pred/conf/test.test']
all_files = glob.glob('*/conf/*.yaml')
ff = 0
for ix, el in enumerate(all_files.copy()):
    if el.find('distribution_target') != -1 and el[el.rfind('/')+1:] != 'emb_valid.yaml':
        all_files.pop(ix + ff)
        ff -= 1
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    start_ix = None
#     yyy = 0
    in_head = {'BatchNorm1d': [False, None],
               'NormEncoder': False,
               'Linear': [False, None],
               'LogSoftmax': [False, None],
               'ReLU': False
              }
    candidates = None
    current_main = None
    yyy = 0
    rnn = transf = False
    input_size = None
    for i, line in enumerate(lines.copy()):
        if line.find('TransfSeqEncoder') != -1:
            transf = True
            input_size = '${pl_module.input_size}'
        elif line.find('RnnSeqEncoder') != -1:
            rnn = True
            input_size = '${pl_module.hidden_size}'
#         if rnn and line.find('hidden_size') != -1:
#             input_size = line[line.find(':') + 2: -1]
#         if transf and line.find('input_size') != -1:
#             input_size = line[line.find(':') + 2: -1]
            
            
        ix = line.find('head_layers:')
        if ix != -1:
            changed = True
            start = True
            start_ix = ix
#             lines[i + yyy] = ' ' * start_ix + 'head:\n'
#             lines.insert(i + yyy, ' ' * (start_ix + 2) + '_target_: ptls.models.Head\n')
#             yyy += 1
        elif start and line[start_ix] != ' ':
            start = False
            break
        elif start:
            lines.pop(i + yyy)
            yyy -= 1
            
            fl = False
            for k, v in in_head.items():
                if line.find(k) != -1:
                    if k == 'BatchNorm1d':
                        if current_main == 'Linear' and \
                           'bias' not in in_head['Linear'][1]:
                            in_head['Linear'][1]['bias'] = [True]
                        if current_main == 'Linear' and \
                           len(in_head['Linear'][1]['in_features']) > len(in_head['Linear'][1]['bias']):
                            in_head['Linear'][1]['bias'] += [True]
                            
                        in_head['BatchNorm1d'][0] = True
                        candidates = {'affine'}
                        current_main = 'BatchNorm1d'
                        fl = True
                    elif k == 'NormEncoder':
                        if current_main == 'Linear' and \
                           'bias' not in in_head['Linear'][1]:
                            in_head['Linear'][1]['bias'] = [True]
                        if current_main == 'Linear' and \
                           len(in_head['Linear'][1]['in_features']) > len(in_head['Linear'][1]['bias']):
                            in_head['Linear'][1]['bias'] += [True]
                            
                        in_head['NormEncoder'] = True
                        candidates = None
                        current_main = 'NormEncoder'
                        fl = True
                    elif k == 'Linear':
                        if current_main == 'Linear' and \
                           'bias' not in in_head['Linear'][1]:
                            in_head['Linear'][1]['bias'] = [True]
                        if current_main == 'Linear' and \
                           len(in_head['Linear'][1]['in_features']) > len(in_head['Linear'][1]['bias']):
                            in_head['Linear'][1]['bias'] += [True]
                            
                        in_head['Linear'][0] = True
                        candidates = {'in_features', 'out_features', 'bias'}
                        current_main = 'Linear'
                        fl = True
                    elif k == 'LogSoftmax':
                        if current_main == 'Linear' and \
                           'bias' not in in_head['Linear'][1]:
                            in_head['Linear'][1]['bias'] = [True]
                        if current_main == 'Linear' and \
                           len(in_head['Linear'][1]['in_features']) > len(in_head['Linear'][1]['bias']):
                            in_head['Linear'][1]['bias'] += [True]
                            
                        in_head['LogSoftmax'][0] = True
                        candidates = None
                        current_main = 'LogSoftmax'
                        fl = True
                    elif k == 'ReLU':
                        if current_main == 'Linear' and \
                           'bias' not in in_head['Linear'][1]:
                            in_head['Linear'][1]['bias'] = [True]
                        if current_main == 'Linear' and \
                           len(in_head['Linear'][1]['in_features']) > len(in_head['Linear'][1]['bias']):
                            in_head['Linear'][1]['bias'] += [True]
                            
                        in_head['ReLU'] = True
                        candidates = None
                        current_main = 'ReLU'
                        fl = True 
            if fl: continue
            
            if not candidates: continue
            for c in candidates:
                if line.find(c) != -1:
                    val = line[line.find(':') + 2: -1]
                    if current_main == 'BatchNorm1d':
                        in_head['BatchNorm1d'][1] = True
                    elif current_main == 'Linear':
                        if in_head['Linear'][1]:
                            if c in in_head['Linear'][1]:
                                in_head['Linear'][1][c] += [val]
                            else:
                                in_head['Linear'][1][c] = [val]
                        else:
                            in_head['Linear'][1] = {c: [val]}  
            
##########################################################################

    changed = start = False
    start_ix = None
    yyy = 0
    for i, line in enumerate(lines.copy()):
        ix = line.find('head_layers:')
        if ix != -1:
            changed = True
            start = True
            start_ix = ix
            lines[i] = ' ' * start_ix + 'head:\n'
            to_add = 1
            lines.insert(i + to_add, ' ' * (start_ix + 2) + '_target_: ptls.models.Head\n')
            to_add += 1
            if in_head['NormEncoder']:
                lines.insert(i + to_add, ' ' * (start_ix + 2) + 'use_norm_encoder: true\n')
                to_add += 1
            if input_size is None:
                input_size = '777'
            lines.insert(i + to_add, ' ' * (start_ix + 2) + f'input_size: {input_size}\n')
            to_add += 1            
            if in_head['Linear'][0]:
                hh = ', '.join(in_head['Linear'][1]['out_features'])
                lines.insert(i + to_add, ' ' * (start_ix + 2) + f'hidden_layers_sizes: [{hh}]\n')
                to_add += 1
#             if in_head['Linear'][1]['bias'][0] == 'false':
#                 lines.insert(i + to_add, ' ' * (start_ix + 2) + 'linear_bias: false\n')
#                 to_add += 1
            if in_head['BatchNorm1d'][0]:
                lines.insert(i + to_add, ' ' * (start_ix + 2) + 'use_batch_norm: true\n')
                to_add += 1
#             if in_head['BatchNorm1d'][1]:
#                 lines.insert(i + to_add, ' ' * (start_ix + 2) + 'batch_norm_affine: false\n')
#                 to_add += 1
            if in_head['LogSoftmax'][0]:
                lines.insert(i + to_add, ' ' * (start_ix + 2) + 'objective: classification\n')
                lines.insert(i + to_add + 1, ' ' * (start_ix + 2) + 'num_classes: 4\n')
                to_add += 2
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

60 files changed


In [533]:
# BatchNorm1d: num_features, affine
# NormEncoder
# Linear: in_features, out_features, bias
# ReLU
# LogSoftmax: dim

15) lr_scheduler

In [534]:
cnt = 0

# all_files = glob.glob('*/conf/cpc_v2_pl_fit_finetuning.yaml')
all_files = glob.glob('*/conf/*.yaml')
        
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = False
    for i, line in enumerate(lines.copy()):
        ix = line.find('lr_scheduler:')
        if ix != -1:
            changed = True
            lines[i] = ' ' * ix + 'lr_scheduler_partial:\n'
            lines.insert(i + 1, ' ' * (ix + 2) + '_partial_: true\n')
            lines.insert(i + 2, ' ' * (ix + 2) + '_target_: torch.optim.lr_scheduler.StepLR\n')
            break
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

87 files changed


16) train block

In [535]:
test_file_name

'scenario_age_pred/conf/test.yaml'

In [536]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    found_start_ix = None
    lr_x = wd_x = None
    ooo = 0
    changed_xxx = start_xxx = False
    found_start_ix_xxx = None
    for i, line in enumerate(lines.copy()):
        ix = line.find('pl_module:')
        if ix != -1:
            start = True
            found_start_ix = ix
            continue
#         if start and (len(line)<found_start_ix or line[found_start_ix] != ' '):
#             start = False
        if start:
            ixxx = line.find('train:')
            if ixxx != -1:
                changed_xxx = start_xxx = True
                found_start_ix_xxx = ixxx
                continue
            if start_xxx and (len(line) <= found_start_ix_xxx or line[found_start_ix_xxx] != ' '):
                start = start_xxx = False
                if lr_x or wd_x:
                    changed = True
                    lines.insert(i + ooo, ' '*found_start_ix_xxx + 'optimizer_partial:\n')
                    lines.insert(i + ooo + 1, ' '*(found_start_ix_xxx+2) + '_partial_: true\n')
                    lines.insert(i + ooo + 2, ' '*(found_start_ix_xxx+2) + '_target_: torch.optim.Adam\n')
                    ooo += 3
                    if lr_x:
                        lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + f'lr: {lr_x}\n')
                        ooo += 1
                    if wd_x:
                        lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + f'weight_decay: {wd_x}\n')
                        ooo += 1             
            
            elif start_xxx:
                uu = line.find('lr:')
                yy = line.find('weight_decay:')
                if uu != -1:
                    lr_x = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
                if yy != -1:
                    wd_x = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

91 files changed


In [537]:
# ContrastiveLoss(sampling_strategy(neg_count), margin),

# BarlowTwinsLoss(sampling_strategy(neg_count), lambd),

# bce,

# NLLLoss

In [538]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    found_start_ix = None
    lr_x = wd_x = None
    ooo = 0
    changed_xxx = start_xxx = False
    found_start_ix_xxx = None
    sampling_strategy = neg_count = margin = lambd = None
    
    for i, line in enumerate(lines.copy()):
        ix = line.find('pl_module:')
        if ix != -1:
            start = True
            found_start_ix = ix
            continue
#         if start and (len(line)<found_start_ix or line[found_start_ix] != ' '):
#             start = False
        if start:
            ixxx = line.find('train:')
            if ixxx != -1:
                changed_xxx = start_xxx = True
                found_start_ix_xxx = ixxx
                continue
            if start_xxx and (len(line) <= found_start_ix_xxx or line[found_start_ix_xxx] != ' '):
                start = start_xxx = False
                if loss_name is not None:
                    lines.insert(i + ooo, ' '*found_start_ix_xxx + 'loss:\n')
                    ooo += 1
                    changed = True
                if loss_name == 'ContrastiveLoss':
                    lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + '_target_: ptls.metric_learn.losses.ContrastiveLoss\n')
                    lines.insert(i + ooo + 1, ' '*(found_start_ix_xxx+2) + f'margin: {margin}\n')
                    lines.insert(i + ooo + 2, ' '*(found_start_ix_xxx+2) + 'sampling_strategy:\n')
                    lines.insert(i + ooo + 3, ' '*(found_start_ix_xxx+4) + '_target_: ptls.metric_learn.sampling_strategies.HardNegativePairSelector\n')
                    lines.insert(i + ooo + 4, ' '*(found_start_ix_xxx+4) + f'neg_count: {neg_count}\n')
                    ooo += 5
                elif loss_name == 'BarlowTwinsLoss':
                    lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + '_target_: ptls.metric_learn.losses.BarlowTwinsLoss\n')
                    lines.insert(i + ooo + 1, ' '*(found_start_ix_xxx+2) + f'lambd: {lambd}\n')
                    lines.insert(i + ooo + 2, ' '*(found_start_ix_xxx+2) + 'sampling_strategy:\n')
                    lines.insert(i + ooo + 3, ' '*(found_start_ix_xxx+4) + '_target_: ptls.metric_learn.sampling_strategies.HardNegativePairSelector\n')
                    lines.insert(i + ooo + 4, ' '*(found_start_ix_xxx+4) + f'neg_count: {neg_count}\n')
                    ooo += 5
                elif loss_name == 'bce':
                    lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + '_target_: torch.nn.BCELoss\n')
                elif loss_name == 'NLLLoss':
                    lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + '_target_: torch.nn.NLLLoss\n')
            
            
            elif start_xxx:
                aa = line.find('loss:')
                if aa != -1:
                    loss_name = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
                bb = line.find('sampling_strategy:')
                if bb != -1:
                    sampl_name = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
                cc = line.find('neg_count:')
                if cc != -1:
                    neg_count = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
                dd = line.find('margin:')
                if dd != -1:
                    margin = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
                ee = line.find('lambd:')
                if ee != -1:
                    lambd = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

91 files changed


In [539]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    found_start_ix = None
    lr_x = wd_x = None
    ooo = 0
    changed_xxx = start_xxx = False
    found_start_ix_xxx = None
    for i, line in enumerate(lines.copy()):
        ix = line.find('pl_module:')
        if ix != -1:
            start = True
            found_start_ix = ix
            continue
#         if start and (len(line)<found_start_ix or line[found_start_ix] != ' '):
#             start = False
        if start:
            ixxx = line.find('train:')
            if ixxx != -1:
                changed_xxx = start_xxx = True
                changed = True
                found_start_ix_xxx = ixxx
                lines.pop(i + ooo)
                ooo -= 1
                continue
            if start_xxx and (len(line) <= found_start_ix_xxx or line[found_start_ix_xxx] != ' '):
                start = start_xxx = False            
            
            elif start_xxx:
                lines.pop(i + ooo)
                ooo -= 1
        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

93 files changed


In [540]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    found_start_ix = None
    lr_x = wd_x = None
    ooo = 0
    changed_xxx = start_xxx = False
    found_start_ix_xxx = None
    n_forward_steps = n_negatives = None
    
    for i, line in enumerate(lines.copy()):
        ix = line.find('pl_module:')
        if ix != -1:
            start = True
            found_start_ix = ix
            continue
#         if start and (len(line)<found_start_ix or line[found_start_ix] != ' '):
#             start = False
        if start:
            ixxx = line.find('cpc:')
            if ixxx != -1:
                changed_xxx = start_xxx = True
                found_start_ix_xxx = ixxx
                lines.pop(i + ooo)
                ooo -= 1
                continue
            if start_xxx and (len(line) <= found_start_ix_xxx or line[found_start_ix_xxx] != ' '):
                start = start_xxx = False
                if loss_name is not None:
                    lines.insert(i + ooo, ' '*found_start_ix_xxx + 'loss:\n')
                    ooo += 1
                    changed = True
                lines.insert(i + ooo, ' '*(found_start_ix_xxx+2) + '_target_: ptls.lightning_modules.cpc_module.CPC_Loss\n')
                lines.insert(i + ooo + 1, ' '*(found_start_ix_xxx+2) + f'n_forward_steps: {n_forward_steps}\n')
                lines.insert(i + ooo + 2, ' '*(found_start_ix_xxx+2) + f'n_negatives: {n_negatives}\n')
      
            elif start_xxx:
                aa = line.find('n_forward_steps:')
                if aa != -1:
                    n_forward_steps = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1
                bb = line.find('n_negatives:')
                if bb != -1:
                    n_negatives = line[line.find(':') + 2: -1]
                    lines.pop(i + ooo)
                    ooo -= 1

        
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

9 files changed


17) add _target_ to all pl_data_modules:

In [541]:
cnt = 0

# all_files = [test_file_name]
all_files = glob.glob('*/conf/*.yaml')
for config_fname in all_files:
    with open(config_fname, 'r') as f:
        lines = f.readlines()
    
    changed = start = False
    ooo = 0
    for i, line in enumerate(lines):
        if line.find('data_module:') != -1:
            if lines[i + ooo + 1].find('_target_') == -1:
                changed = True
                lines.insert(i + ooo + 1, ' '*(line.find('data_module:')+2) + '_target_: ptls.data_load.data_module.cls_data_module.ClsDataModuleTrain\n')
                ooo += 1
    if changed:
        cnt += 1

    with open(config_fname, 'w') as f:
        f.writelines(lines)
        
print(f'{cnt} files changed')

6 files changed
